In [1]:
import json
import pandas as pd
import os
import matplotlib.pyplot as plt
import datetime
%matplotlib inline

In [2]:
# Evaluation

with open ('2009091300.txt') as file:
    data=json.load(file)
    print(data.keys())
    print(data['togo'])

dict_keys(['home', 'away', 'drives', 'scrsummary', 'weather', 'media', 'yl', 'qtr', 'note', 'down', 'togo', 'redzone', 'clock', 'posteam', 'stadium'])
0


In [12]:
mydatadict={}
mydatadict['hometeam']=[]
mydatadict['awayteam']=[]
mydatadict['hometeam_score']=[]
mydatadict['awayteam_score']=[]
mydatadict['gamedate']=[]
mydatadict['gamenumber']=[]


for gametype in ['homestats_','awaystats_']:
    for key in ['totfd', 'totyds', 'pyds', 'ryds', 'pen', 'penyds', 'trnovr', 'pt', 'ptyds', 'ptavg', 'top']:
        mydatadict[gametype + key]=[]

mydatadict['homestats_totfd']=[]
mydatadict['awaystats_totfd']=[]

filenames=[filename for filename in os.listdir() if filename.endswith(".txt")]

for filename in filenames:    
    with open (filename) as file:
        data=json.load(file)
        mydatadict['gamedate'].append(datetime.date(int(filename[:4]),int(filename[4:6]),int(filename[6:8])))
        mydatadict['gamenumber'].append(filename[:-4][-2:])
        mydatadict['hometeam'].append(data['home']['abbr'])
        mydatadict['awayteam'].append(data['away']['abbr'])
        mydatadict['hometeam_score'].append(data['home']['score']['T'])
        mydatadict['awayteam_score'].append(data['away']['score']['T'])
        
        for key in ['totfd', 'totyds', 'pyds', 'ryds', 'pen', 'penyds',
                    'trnovr', 'pt', 'ptyds', 'ptavg', 'top']:
            mydatadict['homestats_'+ key].append(data['home']['stats']['team'][key])
            mydatadict['awaystats_'+ key].append(data['away']['stats']['team'][key])


In [13]:
df=pd.DataFrame(mydatadict)
# df.to_pickle('allgamesdf.pickle')

In [14]:
games2018seasondf=df[df.gamedate.between(datetime.date(2018, 9, 6),datetime.date(2018, 12, 30))]
games2018seasondf.to_pickle('games2018seasondf.pickle')

games2017seasondf=df[df.gamedate.between(datetime.date(2017, 9, 7),datetime.date(2017, 12, 31))]
games2017seasondf.to_pickle('games2017seasondf.pickle')

games2016seasondf=df[df.gamedate.between(datetime.date(2016, 9, 8),datetime.date(2017, 1, 1))]
games2016seasondf.to_pickle('games2016seasondf.pickle')





In [20]:
def GetLastGamesDF(noofgames,team_allgamesdf,teamname):
    
    team_allgamesdf=team_allgamesdf.sort_values(by=['gamedate'],ascending=False).reset_index(drop=True)
    
    trainingsetlists=[]
    
    for gameindex in team_allgamesdf[:-noofgames].index:
        
        trainingsetdict={}
        trainingsetdict['gamenumber']=gameindex
        target_game=team_allgamesdf.loc[gameindex,:]
        if target_game['hometeam']==teamname:
            if target_game['hometeam_score']>target_game['awayteam_score']:
                trainingsetdict['Win']=1
            else:
                trainingsetdict['Win']=0
        else:
            if target_game['hometeam_score']>target_game['awayteam_score']:
                trainingsetdict['Win']=0
            else:
                trainingsetdict['Win']=1
        currentstatsdf=team_allgamesdf[gameindex+1:gameindex+noofgames+1]
        
        
        for gameno,eachgameindex in enumerate(currentstatsdf.index):
                currentgamedf=currentstatsdf.loc[eachgameindex]
    
                if currentgamedf['hometeam']==teamname:
                    trainingsetdict['lastgamenum'+str(gameno)+'-'+'score']=currentgamedf['hometeam_score']
                    trainingsetdict['lastgamenum'+str(gameno)+'-'+'totfd']=currentgamedf['homestats_totfd']
                    trainingsetdict['lastgamenum'+str(gameno)+'-'+'ryds']=currentgamedf['homestats_ryds']
                    trainingsetdict['lastgamenum'+str(gameno)+'-'+'pyds']=currentgamedf['homestats_pyds']
                    trainingsetdict['lastgamenum'+str(gameno)+'-'+'trnovr']=currentgamedf['homestats_trnovr']
                    trainingsetdict['lastgamenum'+str(gameno)+'-'+'top']=currentgamedf['homestats_top']
                    trainingsetdict['lastgamenum'+str(gameno)+'-'+'ptyds']=currentgamedf['homestats_ptyds']
                    
                else:
                    trainingsetdict['lastgamenum'+str(gameno)+'-'+'score']=currentgamedf['awayteam_score']
                    trainingsetdict['lastgamenum'+str(gameno)+'-'+'totfd']=currentgamedf['awaystats_totfd']
                    trainingsetdict['lastgamenum'+str(gameno)+'-'+'ryds']=currentgamedf['awaystats_ryds']
                    trainingsetdict['lastgamenum'+str(gameno)+'-'+'pyds']=currentgamedf['awaystats_pyds']
                    trainingsetdict['lastgamenum'+str(gameno)+'-'+'trnovr']=currentgamedf['awaystats_trnovr']
                    trainingsetdict['lastgamenum'+str(gameno)+'-'+'top']=currentgamedf['awaystats_top']
                    trainingsetdict['lastgamenum'+str(gameno)+'-'+'ptyds']=currentgamedf['awaystats_ptyds']
        
        trainingsetlists.append(trainingsetdict)
        
    return trainingsetlists
        

In [21]:
teamname='KC'
noofgames=3
team_allgamesdf=df[(df.hometeam==teamname) | (df.awayteam==teamname)].sort_values(by=['gamedate'],ascending=False)


pd.DataFrame(GetLastGamesDF(noofgames,team_allgamesdf,teamname))

,Win,gamenumber,lastgamenum0-ptyds,lastgamenum0-pyds,lastgamenum0-ryds,lastgamenum0-score,lastgamenum0-top,lastgamenum0-totfd,lastgamenum0-trnovr,lastgamenum1-ptyds,...,lastgamenum1-top,lastgamenum1-totfd,lastgamenum1-trnovr,lastgamenum2-ptyds,lastgamenum2-pyds,lastgamenum2-ryds,lastgamenum2-score,lastgamenum2-top,lastgamenum2-totfd,lastgamenum2-trnovr
0,1,0,144,265,154,31,24:58,22,2,111,...,37:00,30,1,123,295,174,40,29:14,25,1
1,0,1,111,348,94,27,37:00,30,1,123,...,29:14,25,1,233,212,118,26,28:13,20,0
2,1,2,123,295,174,40,29:14,25,1,233,...,28:13,20,0,63,360,139,37,29:09,27,1
3,1,3,233,212,118,26,28:13,20,0,63,...,29:09,27,1,196,291,49,30,27:19,20,1
4,1,4,63,360,139,37,29:09,27,1,196,...,27:19,20,1,0,353,198,45,33:43,33,1
5,1,5,196,291,49,30,27:19,20,1,0,...,33:43,33,1,51,352,94,40,23:51,18,2
6,1,6,0,353,198,45,33:43,33,1,51,...,23:51,18,2,116,298,126,30,32:43,26,2
7,1,7,51,352,94,40,23:51,18,2,116,...,32:43,26,2,87,307,77,38,30:01,31,0
8,0,8,116,298,126,30,32:43,26,2,87,...,30:01,31,0,100,322,127,42,27:49,24,1
9,1,9,87,307,77,38,30:01,31,0,100,...,27:49,24,1,256,256,106,38,25:34,19,0


In [17]:
team_allgamesdf.T

,169,22,214,90,222,21,207,63,153,86,73,148,95
hometeam,KC,SEA,KC,OAK,KC,CLE,KC,KC,NE,KC,KC,PIT,LAC
awayteam,OAK,KC,BAL,KC,ARI,KC,DEN,CIN,KC,JAX,SF,KC,KC
hometeam_score,35,38,27,33,26,21,30,45,43,30,38,37,28
awayteam_score,3,31,24,40,14,37,23,10,40,14,27,42,38
gamedate,2018-12-30,2018-12-23,2018-12-09,2018-12-02,2018-11-11,2018-11-04,2018-10-28,2018-10-21,2018-10-14,2018-10-07,2018-09-23,2018-09-16,2018-09-09
gamenumber,04,14,05,10,05,03,05,05,12,05,05,05,08
homestats_totfd,18,31,30,28,20,26,20,33,31,26,31,33,33
homestats_totyds,409,464,442,442,330,388,340,551,500,424,384,475,541
homestats_pyds,310,254,348,271,212,286,291,353,327,298,307,442,418
homestats_ryds,99,210,94,171,118,102,49,198,173,126,77,33,123


In [25]:
pd.DataFrame(GetLastGamesDF(noofgames,team_allgamesdf,teamname)).corr()[['Win']].sort_values(by='Win')

,Win
lastgamenum0-totfd,-2.872135e-01
lastgamenum0-trnovr,-2.857143e-01
lastgamenum2-ptyds,-2.819065e-01
lastgamenum1-score,-2.751559e-01
lastgamenum1-totfd,-2.720165e-01
lastgamenum0-pyds,-1.670517e-01
lastgamenum1-ryds,-7.013797e-02
lastgamenum2-ryds,-2.049198e-02
lastgamenum0-ptyds,-8.471731e-03
gamenumber,-9.663251e-18
